In [1]:
import json
import random
import os
from glob import glob

In [3]:
files = glob('/home/husein/ssd3/gov.my/ultrachat-*.jsonl')
files = [f for f in files if 'glaive_coder_raw_text' not in f]
files

['/home/husein/ssd3/gov.my/ultrachat-jurnaldbp.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-jurnaldbp-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-maktabahalbakri.com.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-gov.my.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-crossref-melayu.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-lom-agc.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-astroawani-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-textbooks.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-crossref-melayu-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-epenerbitan-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-hansard-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-ms-wikipedia.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-muftiwp.gov.my.texts.jsonl']

In [4]:
for f in files:
    new_f = f'pair-{os.path.split(f)[-1]}'
    data = []
    with open(f) as fopen:
        for l in fopen:
            data.append(json.loads(l))
    
    indices = {i for i in range(len(data))}
    pairs = []
    for i in range(len(data)):
        l = data[i]
        context = l[0]['content']
        pos = []
        for l_ in l[1:]:
            v = list(l_.values())
            v = [v_ for v_ in v if v_ is not None and v_ not in {'user', 'assistant'}]
            pos.extend(v)
        pos = list(set(pos))

        neg_i = random.choice(list(indices - {i}))
        neg = []
        for l_ in data[neg_i][1:]:
            v = list(l_.values())
            v = [v_ for v_ in v if v_ is not None and v_ not in {'user', 'assistant'}]
            neg.extend(v)

        neg = list(set(neg))

        pairs.append({
            'query': context,
            'pos': pos,
            'neg': neg
        })

        l = l[1:]
        inputs = []
        for no, r in enumerate(l):
            if r['content_ms'] is None:
                break

            inputs.append(r['content_ms'])

        if len(inputs) % 2 != 0:
            inputs = inputs[:-1]

        for k in range(0, len(inputs), 2):
            neg_i = random.choice(list(indices - {i}))
            
            if random.random() > 0.3:
                continue

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k + 1]],
                'neg': [data[neg_i][0]['content']]
            })
        
        for k in range(1, len(inputs), 2):
            neg_i = random.choice(list(indices - {i}))
            
            if random.random() > 0.3:
                continue

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k - 1]],
                'neg': [data[neg_i][0]['content']]
            })
            
        l = l[1:]
        inputs = []
        for no, r in enumerate(l):
            if r['content'] is None:
                break

            inputs.append(r['content'])

        if len(inputs) % 2 != 0:
            inputs = inputs[:-1]

        for k in range(0, len(inputs), 2):
            
            if random.random() > 0.3:
                continue

            neg_i = random.choice(list(indices - {i}))

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k + 1]],
                'neg': [data[neg_i][0]['content']]
            })
        
        for k in range(1, len(inputs), 2):
            
            if random.random() > 0.3:
                continue
                
            neg_i = random.choice(list(indices - {i}))

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k - 1]],
                'neg': [data[neg_i][0]['content']]
            })
            
    with open(new_f, 'w') as fopen:
        for l in pairs:
            if not len(l['pos']) or not len(l['neg']):
                continue
            fopen.write(f'{json.dumps(l)}\n')

In [6]:
files = glob('pair-ultrachat-*.jsonl')
files

['pair-ultrachat-crossref-melayu.jsonl',
 'pair-ultrachat-textbooks.jsonl',
 'pair-ultrachat-muftiwp.gov.my.texts.jsonl',
 'pair-ultrachat-epenerbitan-malay.jsonl',
 'pair-ultrachat-gov.my.jsonl',
 'pair-ultrachat-lom-agc.jsonl',
 'pair-ultrachat-ms-wikipedia.jsonl',
 'pair-ultrachat-jurnaldbp.jsonl',
 'pair-ultrachat-astroawani-malay.jsonl',
 'pair-ultrachat-crossref-melayu-malay.jsonl',
 'pair-ultrachat-jurnaldbp-malay.jsonl',
 'pair-ultrachat-hansard-malay.jsonl',
 'pair-ultrachat-maktabahalbakri.com.jsonl']

In [7]:
from huggingface_hub import HfApi
api = HfApi()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [8]:
for f in files:
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/instructions-pair-mining',
        repo_type='dataset',
    )

pair-ultrachat-crossref-melayu.jsonl


pair-ultrachat-crossref-melayu.jsonl:   0%|          | 0.00/211M [00:00<?, ?B/s]

pair-ultrachat-textbooks.jsonl


pair-ultrachat-textbooks.jsonl:   0%|          | 0.00/5.54G [00:00<?, ?B/s]

pair-ultrachat-muftiwp.gov.my.texts.jsonl


...

pair-ultrachat-jurnaldbp-malay.jsonl:   0%|          | 0.00/606M [00:00<?, ?B/s]

pair-ultrachat-hansard-malay.jsonl


pair-ultrachat-hansard-malay.jsonl:   0%|          | 0.00/3.57G [00:00<?, ?B/s]

pair-ultrachat-maktabahalbakri.com.jsonl


pair-ultrachat-maktabahalbakri.com.jsonl:   0%|          | 0.00/431M [00:00<?, ?B/s]